In [13]:
import os
import torch
import torchvision
import torch.nn as nn
import pickle
import pylab
import numpy as np
import scipy
import torch.optim as optim
import pandas as pd
import torchvision.datasets as datasets
import time
import tensorflow as tf
import alibi


from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Input, UpSampling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import MinMaxScaler

from scipy.spatial.distance import euclidean
from scipy.stats import shapiro, normaltest

from torchvision import transforms
from torchvision.utils import save_image

from collections import Counter

from copy import deepcopy

from torch.autograd import Variable

from skimage.io import imread

# Local imports
from local_models import *
from helper_functions import *
from piece_hurdle_model import *
from optimize_explanations import *
from evaluation_metrics import *

#new lib
import matplotlib.pyplot as plt
import glob
from sklearn.metrics import accuracy_score
from itertools import combinations
from sklearn.model_selection import train_test_split
from PIL import Image
import csv

In [14]:
# Load models and data
G, cnn = load_models(CNN, Generator)

# train_loader, test_loader = load_dataloaders()
# X_train, y_train, X_test, y_test = get_MNIST_data(datasets)

train_loader, test_loader = load_fashion_dataloaders()
X_train, y_train, X_test, y_test = get_FashionMNIST_data(datasets)

expt1_data = pd.DataFrame(columns=['rand_num', 'name', 'MC-Mean', 'MC-STD', 'NN-Dist', 'IM1', 'IM2', 'optim_time'])

df_piece = pd.DataFrame(columns=['rand_num', 'name', 'MC-Mean', 'MC-STD', 'NN-Dist', 'IM1', 'IM2', 'optim_time'])
df_ME = pd.DataFrame(columns=['rand_num', 'name', 'MC-Mean', 'MC-STD', 'NN-Dist', 'IM1', 'IM2', 'optim_time'])
df_CME = pd.DataFrame(columns=['rand_num', 'name', 'MC-Mean', 'MC-STD', 'NN-Dist', 'IM1', 'IM2', 'optim_time'])


In [15]:
cnn

CNN(
  (main): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Dropout2d(p=0.1, inplace=False)
    (4): Conv2d(8, 16, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): Dropout2d(p=0.1, inplace=False)
    (8): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Dropout2d(p=0.1, inplace=False)
    (12): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU(inplace=True)
    (15): Dropout2d(p=0.2, inplace=False)
    (16): Conv2d(64, 128, kernel_size=(

In [16]:
def get_activations(data):
    activations_list = []
    for i in range(len(data)):
        
        pred, activations = cnn(data[i])
        activations_list.append(activations)
    return np.array(activations_list)


# X_train_act = get_activations(X_train)
# Y_train_act = get_activations(y_train)
# X_test_act = get_activations(X_test)
# y_test_act = get_activations(y_test)


# X_train_act = np.load("data/distribution_data/X_train_act.npy")
# X_test_act = np.load("data/distribution_data/X_test_act.npy")
# X_train_pred = np.load("data/distribution_data/X_train_pred.npy")
# X_test_pred = np.load("data/distribution_data/X_test_pred.npy")
# k_nn = KNeighborsClassifier(n_neighbors=1, algorithm='brute')
# k_nn.fit(X_train_act, X_train_pred)

In [17]:
def get_label_and_pred():
    folder_path = './data/z_Generator/incorrect_latent'

    labels_list = []
    predicted_list = []
    filename_list = []
    pt_list = []

    
    for filename in os.listdir(folder_path):
        
        parts = filename.split('_')  
        labels = parts[2]  
        predicted = parts[5]


        pt_list.append(filename)
        png_filename = filename.replace('.pt', '.png')
        filename_list.append(png_filename)
        labels_list.append(labels)
        predicted_list.append(predicted)
    
    return labels_list,predicted_list,filename_list,pt_list

labels_list,predicted_list,filename_list,pt_list = get_label_and_pred()


image_folder_path = './data/incorrect_img'
pt_folder_path = './data/z_Generator/incorrect_latent_z'



In [18]:
def reverse_output_label(category_name):
    
    output_mapping = {
        "T-shirt": 0,
        "Trouser": 1,
        "Pullover": 2,
        "Dress": 3,
        "Coat": 4,
        "Sandal": 5,
        "Shirt": 6,
        "Sneaker": 7,
        "Bag": 8,
        "Ankle Boot": 9
    }

    
    return output_mapping[category_name]



In [19]:
# Probabilitiy threshold for identifying "Exceptional Features" with PIECE
alpha = 0.15

In [20]:
def load_png_as_mnist_format(png_path):
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5))
    ])
    
    image = Image.open(png_path)
    mnist_tensor = torch.unsqueeze(transform(image), dim=1)
    
    return mnist_tensor

In [21]:
# Iterate though 41 Incorrect examples from MNIST
# Iterate though 40 Incorrect examples from Fashion-MNIST
for rand_num in range(0, 40):

	"""
    Similar to using the mnist dataset, images that were misclassified by cnn were selected and the labels and predictions were loaded.
    """
	image_name = filename_list[rand_num]
	image_folder_path = './data/incorrect_img/'+image_name
	img = Image.open(image_folder_path)
	
	original_query_img = load_png_as_mnist_format(image_folder_path)
	
	z = torch.load("data/z_Generator/incorrect_latent_z/" + pt_list[rand_num],map_location='cpu') 
	
	original_query_pred = predicted_list[rand_num]
	#print(original_query_pred)
	original_query_pred = reverse_output_label(original_query_pred)
	#print(original_query_pred)

	target_class = labels_list[rand_num]
	target_class = reverse_output_label(target_class)
	formatted_integer = np.array([target_class])
	target_class = formatted_integer.item()
	#z = torch.load("data/z_Generator/incorrect_latent_z/" + pt_list[rand_num],map_location='cpu') 
	query_activations = cnn(G(z))[1][0]
	# print("image_name",image_name)
	# print("original_query_pred",original_query_pred)
	# print("target_class",target_class)
	# print("pt",pt_list[rand_num])

	# fig, axarr = plt.subplots(1, 2, figsize=(10, 5))

	# axarr[0].imshow(img)
	# axarr[0].set_title('Original Image')
	# axarr[0].axis('off')  

	
	# axarr[1].imshow(G(z).detach().numpy().reshape(28, 28), cmap='gray')
	# axarr[1].set_title('Generated Image G(z)')
	# axarr[1].axis('off')  

	# plt.show()

	#### ========== First two steps of PIECE Algorithm ========== ####
	# Step 1: Acquire the probability of each features, and identify the excpetional ones (i.e., those with a probability lower than alpha)
	df = acquire_feature_probabilities(target_class, cnn, original_query_img, alpha) 
	# Step 2: Filter out exceptional features which we want to change, and change them to their expected values in the counterfactual class
	df = filter_df_of_exceptional_noise(df, target_class, cnn, alpha)
	# Sort by least probable to the most probable
	df = df.sort_values('Probability of Event')
	# Get x' -- The Ideal Explanation
	ideal_xp = modifying_exceptional_features(df, target_class, query_activations)   
	ideal_xp = ideal_xp.clone().detach().float().requires_grad_(False)

	save_paths = {
		'PIECE': 'data/PIECE/a1/',
		'Min-Edit': 'data/Min-Edit/',
		'C-Min-Edit': 'data/C-Min-Edit/'
	}

	# For Save z.pt
	for algorithm, path in save_paths.items():
		os.makedirs(path, exist_ok=True)


	for name in ['PIECE', 'Min-Edit', 'C-Min-Edit']:  # 'CEM', 'Proto-CF']:

		# print(" ")
		# print("-------------------------------")
		# print(rand_num, name)
		# print("-------------------------------")

		cnn = cnn.eval()
		temp_data = pd.DataFrame(columns=['rand_num', 'name', 'MC-Mean', 'MC-STD', 'NN-Dist', 'IM1', 'IM2', 'optim_time'])

		# Query
		x_q = cnn(G(z))[1][0]

		# Explanation latent input (to optimize...)
		z_e = z.clone().detach().float().requires_grad_()

		criterion = nn.MSELoss()

		start_time = time.time()

		if name == 'PIECE':
			optimizer = optim.Adam([z_e], lr=0.01)
			z_e = optim_PIECE(G, cnn, ideal_xp, z_e, criterion, optimizer)

		elif name == 'Min-Edit':
			optimizer = optim.Adam([z_e], lr=0.001)
			z_e = optim_min_edit(cnn, G, z_e, optimizer, target_class)

		elif name == 'C-Min-Edit':
			optimizer = optim.Adam([z_e], lr=0.001)
			z_e = optim_c_min_edit(G, cnn, x_q, z_e, criterion, optimizer, target_class)

		
		torch.save(z_e, os.path.join(save_paths[name], f'z_e_{rand_num}.pt'))

		optim_time = time.time() - start_time

		if name == 'PIECE' or name == 'Min-Edit' or name == 'C-Min-Edit':
			I_e = G(z_e)
			
		elif name == 'CEM' or name == 'Proto-CF':
			I_e = torch.tensor(xp, dtype=torch.float32).reshape(-1,1,28,28)

		save_name = name
		imgnum = rand_num 
		save_query_and_gan_xp_for_final_data(I_e, cnn, z, G, z_e, original_query_img, save_name, imgnum, target_class, original_query_pred)

		# New prediction of explanation
		new_pred = int(torch.argmax(torch.exp(  cnn(I_e)[0]  )))
		
		# Metrics for Plausibility
		mc_dropout_results = mc_dropout(cnn, new_pred, I_e)
		#nn_dist, _ = k_nn.kneighbors(X=np.array(    cnn(I_e)[1].detach().numpy()  )  , n_neighbors=2)
		# IM1 = IM1_metric(I_e, aes, original_query_pred, new_pred)
		# IM2 = IM2_metric(I_e, aes, ae_full, new_pred)
		
	
		temp_data = pd.Series({
			'rand_num': rand_num,
			'name': name,
			'MC-Mean': mc_dropout_results.mean(),
			'MC-STD': mc_dropout_results.std(),
			#'NN-Dist': nn_dist[0][0],
			# 'IM1': IM1,
			# 'IM2': IM2,
			'optim_time': optim_time
		})

		# 将该 Series 添加到 DataFrame 中
		expt1_data = pd.concat([expt1_data, temp_data.to_frame().T], ignore_index=True) 
		if name == 'PIECE':
			df_piece = pd.concat([df_piece, temp_data.to_frame().T], ignore_index=True) 
		elif name == 'Min-Edit':
			df_ME = pd.concat([df_ME, temp_data.to_frame().T], ignore_index=True) 
		elif name == 'C-Min-Edit':
			df_CME = pd.concat([df_CME, temp_data.to_frame().T], ignore_index=True)
		
		
		

	

	

	

c:\Users\13478\Desktop\wj\helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['flag'] = np.zeros(df.shape[0])
c:\anaconda\envs\IML\lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
c:\Users\13478\Desktop\wj\helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['flag'] = np.zeros(df.shape[0])
c:\anaconda\envs\IML\lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid valu

In [23]:
expt1_data.to_csv('ablation1.csv', index=False)

# df_piece.to_csv('df_piece.csv', index=False)
# df_ME.to_csv('df_ME.csv', index=False)
# df_CME.to_csv('df_CME.csv', index=False)
	


New Evalution 

In [24]:


def generator_Classifier():
    """
    one-Nearest Neighbor Classifier
    Determine how close the modified image is to the original.
    When the accuracy approaches 50%, it indicates that the distance between generated and real samples is close, without being caused by overfitting.
    """
    
    file_paths_folder1 = glob.glob("data/z_Generator/incorrect_latent_z/*.pt")
    file_paths_folder2 = glob.glob("data/z_Generator/correct_latent_z/*.pt")
    #file_paths_folder3 = glob.glob("data/PIECE/a1/*.pt")
    #file_paths_folder3 = glob.glob("data/Min-Edit/*.pt")
    file_paths_folder3 = glob.glob("data/C-Min-Edit/*.pt")

    
    all_file_paths = file_paths_folder1 + file_paths_folder2

    
    images = []
    labels = []

    # Loop through each file path
    for file_path in all_file_paths:
        
        z = torch.load(file_path) 
        image = G(z)
        images.append(image)
        labels.append(torch.tensor(1))  
    
    for file_path in file_paths_folder3:
        
        z = torch.load(file_path)
        image = G(z)
        images.append(image)
        labels.append(torch.tensor(0))  
    
    D = list(zip(images, labels))

     # Initialize KNN classifier
    knn = KNeighborsClassifier(n_neighbors=1)

    accuracies = []

    # Leave-one-out cross-validation
    for i in range(len(D)):
        # Remove the ith sample from the dataset as the test sample
        D_train = D[:i] + D[i+1:]
        D_test = [D[i]]

        X_train, y_train = zip(*D_train)
        X_test, y_test = zip(*D_test)

        X_train_np = [x.detach().numpy().flatten() for x in X_train]
        y_train_np = [y.detach().numpy().flatten() for y in y_train]
        X_train_flat = [x.flatten() for x in X_train]
        X_test_flat = [x.detach().numpy().flatten() for x in X_test]
        y_test_flat = [y.detach().numpy().flatten() for y in y_test]
        
        knn.fit(X_train_np, y_train_np)
        
        # Classify the test sample
        y_pred = knn.predict(X_test_flat)

        
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    
    mean_accuracy = sum(accuracies) / len(accuracies)

    
    print("Mean Accuracy:", mean_accuracy)

    return mean_accuracy

generator_Classifier()



c:\anaconda\envs\IML\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\anaconda\envs\IML\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\anaconda\envs\IML\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\anaconda\envs\IML\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

Mean Accuracy: 0.31724137931034485


0.31724137931034485